# Getting Started with RL4UC

This notebook will briefly describe how to use RL4UC. It will cover the following elements:

- Making an environment
- Interacting with the environment
- Training and testing an agent

## Making an Environment

Environments are specified with just a handful of arguments. The key ones are: 

- `num_gen`: number of controllable generators (must be at least 5)
- `dispatch_freq_mins`: length of settlement periods/frequency of decision-making (every 30 minutes by default)
- `voll`: the 'value of lost load', determining the magnitude of the penalty for failing to meet demand
- `arma_demand` and `arma_wind`: a dictionary specifying the details of the auto-regressive moving average processes determining the forecast errors for demand and wind
- `usd_per_kgco2`: the cost in USD of emitting a kilogram of CO2

You can manually create an environment with your specifications using `make_env()`, or you can create an environment from a json file using `make_env_from_json()`. The json which must be located in `data/envs`. There are already a few datasets in there for systems ranging from 5 to 30 generators. 

First we will create an environment manually. All the variables have default values, so we will just specify a few here:

In [1]:
from rl4uc.environment import make_env
import numpy as np

env = make_env()

The generator specifications are all derived from a single dataset, widely used in the UC literature, from: Kazarlis, S.A., Bakirtzis, A.G. and Petridis, V., 1996. A genetic algorithm solution to the unit commitment problem. IEEE transactions on power systems, 11(1), pp.83-92.

For 10 generators, the paper specifies quadratic fuel cost curves, cold and hot start costs (RL4UC currently considers hot starts), minimum up/down times and minimum and maximum operating outputs. 

Here is the data:

In [2]:
env.gen_info

,min_output,max_output,status,a,b,c,t_min_down,t_min_up,t_max_up,hot_cost,cold_cost,cold_hrs,kgco2_per_mmbtu,usd_per_mmbtu,fuel_type,outage_rate,outage_weibull_shape,max_cost_per_mwh,min_fuel_cost
0,150,455,16,0.00048,16.19,1000.0,16,16,1000000,4500,9000,1000000,95,1.30,coal,0.001,8.0,22.928667,18.606202
2,20,130,-10,0.00200,16.60,700.0,10,10,1000000,550,1100,1000000,54,2.69,gas,0.005,3.0,51.640000,22.244615
4,25,162,-12,0.00398,19.70,450.0,12,12,1000000,900,1800,1000000,54,2.69,gas,0.005,1.3,37.799500,23.122538
6,25,85,-6,0.00079,27.74,480.0,6,6,1000000,260,520,1000000,54,2.69,gas,0.001,1.0,46.959750,33.454209
8,10,55,-2,0.00222,27.27,665.0,2,2,1000000,30,60,1000000,73,3.18,oil,0.001,1.7,93.792200,39.483009


When we set `num_gen < 10`, RL4UC takes a subset of these generators by default. When using more than 10 generators, we use duplicates of these generators.

如果使用make env 的方法 就是用默认的机组参数进行复制和采样 如果用是make from json 那就是自己自定义机组参数

When making an environment from a json, we need to specify the environment's 'name', that is the prefix to '.json'. As mentioned, this must be in the directory `data/envs`. 

In [3]:
from rl4uc.environment import make_env_from_json

env = make_env_from_json('10gen_carbon1')
env.gen_info

,min_output,max_output,status,a,b,c,t_min_down,t_min_up,t_max_up,hot_cost,cold_cost,cold_hrs,kgco2_per_mmbtu,usd_per_mmbtu,fuel_type,outage_rate,outage_weibull_shape,max_cost_per_mwh,min_fuel_cost
0,150,455,16,0.002234,75.345769,4653.846154,16,16,1000000,4500,9000,1000000,95,1.30,coal,0.001,8.0,106.706487,86.590403
1,150,455,16,0.001443,80.325385,4514.230769,16,16,1000000,5000,10000,1000000,95,1.30,coal,0.002,12.0,110.636660,90.903196
2,20,130,-10,0.004007,33.261710,1402.602230,10,10,1000000,550,1100,1000000,54,2.69,gas,0.005,3.0,103.471970,44.571925
3,20,130,-10,0.004228,33.061338,1362.527881,10,10,1000000,560,1120,1000000,54,2.69,gas,0.010,1.1,101.272289,44.091942
4,25,162,-12,0.007975,39.473234,901.672862,12,12,1000000,900,1800,1000000,54,2.69,gas,0.005,1.3,75.739519,46.331033
5,20,80,-6,0.014266,44.602751,741.375465,6,6,1000000,170,340,1000000,54,2.69,gas,0.002,4.0,81.956854,55.011262
6,25,85,-6,0.001583,55.583123,961.784387,6,6,1000000,260,520,1000000,54,2.69,gas,0.001,1.0,94.094072,67.032783
7,10,55,-2,0.008870,55.670943,1417.547170,2,2,1000000,30,60,1000000,73,3.18,oil,0.004,2.0,197.514364,81.932401
8,10,55,-2,0.004768,58.570472,1428.286164,2,2,1000000,30,60,1000000,73,3.18,oil,0.001,1.7,201.446769,84.801557
9,10,55,-2,0.003716,59.687327,1439.025157,2,2,1000000,30,60,1000000,73,3.18,oil,0.010,1.5,203.627000,86.055784


The preset environments are aimed at better defining benchmarks that people can compare performance on.\
预设环境旨在更好地定义人们可以比较性能的基准。

### Training and Testing Modes

Environments have two modes: train and test. Training mode is designed for when we want to sample from a **range of episodes**, specified in `env.profiles_df`. In training mode, when we 'reset' with `env.reset()` the environment samples a new episode from `env.profiles_df`. This makes it useful for training agents. In test mode, we are considering the case where `env.profiles_df` is **a single test episode**, and we want to evaluate the performance. Typically the test problem will not have been observed in training.\
想要写test 只需要把预定义的文件改为 想要测试的某一天或者 某一个时间段 同时  test的数据 在训练集里不应该被agent跑到过  

When making an environment in training mode (e.g. `make_env(mode='train')`), the profiles in `data/train_data_10gen.csv` are automatically scaled to the number of generators and available as episodes. These profile are based on data from the GB power system. By contrast, when using a test environment you **must** specify a test profile.

Some other differences between training and testing:

- In train mode, states are terminal if there is lost load. In testing the episode continues. 
- In train mode, resetting the environment randomly initialises the generator up/down times. In test mode they are initialised to `env.gen_info.status`
env.gen_info.status

In [4]:
env.gen_info.status

0    16
1    16
2   -10
3   -10
4   -12
5    -6
6    -6
7    -2
8    -2
9    -2
Name: status, dtype: int64

## Interacting with the environment

RL4UC **roughly** follows the OpenAI Gym API. In other words, we act on the environment by executing `env.step(action)`, which returns an observation, reward and an indicator of whether the state is terminal. (Currently the 'info' return is not implemented). We can reset the environment to begin a new episode with `env.reset()`. **An action is a binary sequence of length `env.num_gen`, where 1 indicates turning the generator on, and 2 indicates turning it off.**

For instance:

In [5]:
env = make_env_from_json('5gen') # make the environment
obs = env.reset() # reset the environment
action = np.array([1,1,1,1,0]) # turn all but the last generator on
obs, reward, done = env.step(action) # act on the environment
#print(obs)
print("Reward: {}".format(reward))
print("Done: {}".format(done))
print("obs:{}".format(obs))

Reward: -6166.595023577391
Done: False
obs:{'status': array([17,  1,  1,  1, -3]), 'demand_forecast': array([397.52622545, 392.15947636, 391.04261236, 392.65263709,
       388.79437964, 382.499328  , 373.02774109, 368.53127564,
       366.05096727, 359.45131636, 361.78657745, 363.80273455,
       376.63941818, 388.31572364, 405.02516945, 414.43873745,
       425.99900509, 429.58167273, 435.64464873, 435.31104   ,
       435.36905891, 432.39558982, 428.94346473, 430.29240436,
       428.11669527, 424.49051345, 419.74746764, 415.75866764,
       415.86020073, 414.43873745, 417.47022545, 422.66291782,
       433.58497745, 447.36446836, 461.158464  , 470.35446109,
       475.89526691, 477.99845236, 477.12816873, 476.635008  ,
       472.44314182, 471.64538182, 472.21106618, 466.583232  ,
       461.70964364, 450.51199418, 427.75407709, 408.79639855]), 'demand_errors': array([12.6763257,  0.       ,  0.       ,  0.       ,  0.       ]), 'wind_forecast': array([ 61.05127716,  92.54398895, 10

We can see the reward achieved by the action. In addition, we can see whether the environment is 'done': that is, whether it has reached a terminal state. 

**NOTE: currently (and counter-intuitively) you can still continue to act on an environment after it has reached a terminal state. When training, you should embed each episode in a while loop, that checks for if environment is done (as in the next section).**
**注意：目前（并且违反直觉）您仍然可以在环境达到最终状态后继续对其进行操作。训练时，您应该将每一集嵌入到一个 while 循环中，以检查环境是否已完成（如下一节所述）。**

### States

States are comprised predominantly of the following components: generator up/down times, demand forecast, wind forecast, timestep. In addition are the forecast errors, produced by the ARMA processes. **Note: while the forecast errors are returned in the observation, you may want to consider whether the agent *should* observe this in a particular problem setting. If you are training an agent to solve the day-ahead problem, it probably *shouldn't* observe them, as that won't be possible when it comes to testing!**

Let's see the observed state from the code block above:
状态主要由以下部分组成：发电机启动/停机时间、需求预测、风力预测、时间步长。此外还有由 ARMA 过程产生的预测误差。注意：虽然在观察中返回了预测误差，但您可能需要考虑代理是否应该在特定问题设置中观察这一点。如果你正在训练一个代理来解决前一天的问题，它可能不应该观察它们，因为这在测试中是不可能的！

In [6]:
obs

{'status': array([17,  1,  1,  1, -3]),
 'demand_forecast': array([397.52622545, 392.15947636, 391.04261236, 392.65263709,
        388.79437964, 382.499328  , 373.02774109, 368.53127564,
        366.05096727, 359.45131636, 361.78657745, 363.80273455,
        376.63941818, 388.31572364, 405.02516945, 414.43873745,
        425.99900509, 429.58167273, 435.64464873, 435.31104   ,
        435.36905891, 432.39558982, 428.94346473, 430.29240436,
        428.11669527, 424.49051345, 419.74746764, 415.75866764,
        415.86020073, 414.43873745, 417.47022545, 422.66291782,
        433.58497745, 447.36446836, 461.158464  , 470.35446109,
        475.89526691, 477.99845236, 477.12816873, 476.635008  ,
        472.44314182, 471.64538182, 472.21106618, 466.583232  ,
        461.70964364, 450.51199418, 427.75407709, 408.79639855]),
 'demand_errors': array([12.6763257,  0.       ,  0.       ,  0.       ,  0.       ]),
 'wind_forecast': array([ 61.05127716,  92.54398895, 100.18222821,  96.28115323,
   

The `status` component gives the up/down times in decision periods. Remember that the generators have minimum up/down time constraints, so the up/down times are a very important component of the state vector.

## Training an Agent

Now we'll look at how to train an agent. 

In this case we'll look at a simple Q-learning agent. Needless to say, this is a pretty simple solution that could be vastly improved.

**Note: this requires Pytorch to run yourself.**

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim

class QAgent(nn.Module):
    def __init__(self, env):
        super(QAgent, self).__init__()
        self.num_gen = env.num_gen
        
        self.num_nodes = 32
        self.gamma = 0.99
        self.activation = torch.tanh
        
        # There are 2N output nodes, corresponding to ON/OFF for each generator
        self.n_out = 2*self.num_gen
        
        self.obs_size = self.process_observation(env.reset()).size
        
        self.in_layer = nn.Linear(self.obs_size, self.num_nodes)
        self.out_layer = nn.Linear(self.num_nodes, self.n_out) 
        
        self.optimizer = optim.Adam(self.parameters(), lr=3e-04)
        self.criterion = nn.MSELoss()
        
                
    def process_observation(self, obs):
        """
        Process an observation into a numpy array.
        
        Observations are given as dictionaries, which is not very convenient
        for function approximation. Here we take just the generator up/down times
        and the timestep.
        
        Customise this!
        """
        obs_new = np.concatenate((obs['status'], [obs['timestep']]))
        return obs_new
    
    def forward(self, obs):
        x = torch.as_tensor(obs).float()
        x = self.activation(self.in_layer(x))
        return self.out_layer(x)
        
    def act(self, obs):
        """
        Agent always acts greedily w.r.t Q-values!
        """
        processed_obs = self.process_observation(obs)

        q_values = self.forward(processed_obs)
        q_values = q_values.reshape(self.num_gen, 2)
        action = q_values.argmax(axis=1).detach().numpy()
        
        return action, processed_obs
    
    def update(self, memory, batch_size=None):
        
        if batch_size == None:
            batch_size = memory.capacity
        
        data = memory.sample(batch_size)
        
        qs = self.forward(data['obs']).reshape(batch_size, self.num_gen, 2)
        
        # A bit of complicated indexing here! 
        # We are using the actions [batch_size, num_gen] to index Q-values
        # which have shape [batch_size, num_gen, 2]
        m,n = data['act'].shape
        I,J = np.ogrid[:m,:n]
        qs = qs[I, J, data['act']]
        
        next_qs = self.forward(data['next_obs']).reshape(batch_size, self.num_gen, 2)
        next_acts = next_qs.argmax(axis=2).detach().numpy()
        
        # The same complicated indexing! 
        m,n = next_acts.shape
        I,J = np.ogrid[:m,:n]
        next_qs = next_qs[I, J, next_acts]
        
        # Recasting rewards into the same shape as next_qs
        m,n = next_qs.shape
        rews = np.broadcast_to(data['rew'], (self.num_gen,batch_size)).T
        rews = torch.as_tensor(rews).float()

        td_target = rews + self.gamma * next_qs
                
        criterion = nn.MSELoss()
        loss = criterion(qs, td_target)
        
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

class ReplayMemory(object):

    def __init__(self, capacity, obs_size, act_dim):
        
        self.capacity = capacity
        self.obs_size = obs_size
        self.act_dim = act_dim 
        
        self.act_buf = np.zeros((self.capacity, self.act_dim))
        self.obs_buf = np.zeros((self.capacity, self.obs_size))
        self.rew_buf = np.zeros(self.capacity)
        self.next_obs_buf = np.zeros((self.capacity, self.obs_size))
        
        self.num_used = 0
        
    def store(self, obs, action, reward, next_obs):
        """Store a transition in the memory"""
        idx = self.num_used % self.capacity
        
        self.act_buf[idx] = action
        self.obs_buf[idx] = obs
        self.rew_buf[idx] = reward
        self.next_obs_buf[idx] = next_obs
        
        self.num_used += 1
    
    def sample(self, batch_size):
        idx = np.random.choice(np.arange(self.capacity), size=batch_size, replace=False)
        
        data = {'act': self.act_buf[idx],
                'obs': self.obs_buf[idx],
                'rew': self.rew_buf[idx],
                'next_obs': self.next_obs_buf[idx]}
        
        return data
        
    def is_full(self):
        return (self.num_used >= self.capacity)
    
    def reset(self):
        self.num_used = 0 
        
def train():
    
    MEMORY_SIZE = 200
    N_EPOCHS = 500
    
    env = make_env_from_json('5gen')
    agent = QAgent(env)
    memory = ReplayMemory(MEMORY_SIZE, agent.obs_size, env.num_gen)
    
    log = {'mean_timesteps': [],
           'mean_reward': []}
    
    for i in range(N_EPOCHS):
        if i % 10 == 0:
            print("Epoch {}".format(i))
        epoch_timesteps = []
        epoch_rewards = []
        while memory.is_full() == False:
            done = False
            obs = env.reset()
            timesteps = 0
            while not done: 
                action, processed_obs = agent.act(obs)
                next_obs, reward, done = env.step(action)
                
                next_obs_processed = agent.process_observation(next_obs)
                
                memory.store(processed_obs, action, reward, next_obs_processed)
                
                obs = next_obs
                
                if memory.is_full():
                    break
                
                timesteps += 1
                if done:
                    epoch_rewards.append(reward)
                    epoch_timesteps.append(timesteps)
                    
        log['mean_timesteps'].append(np.mean(epoch_timesteps))
        log['mean_reward'].append(np.mean(epoch_rewards))
        
        agent.update(memory)
        memory.reset()
                    
    return agent, log


The following cell trains the agent. Note that this might take a few minutes to run.

In [ ]:
agent, log = train()

Epoch 0


/var/folders/f6/spy8ryks12zbx47dctnl5x800000gn/T/ipykernel_40712/1561924491.py:83: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /tmp/pip-req-build-rbnx5dkf/torch/csrc/utils/tensor_numpy.cpp:143.)
  rews = torch.as_tensor(rews).float()


And now let's see if the agent managed to improve its performance: 

In [ ]:
import pandas as pd 

pd.Series(log['mean_reward']).rolling(50).mean().plot()

Some things you may want to consider when writing an agent:

- The action space is combinatorial
- Illegal actions are corrected by the environment
- Observations and rewards are unnormalised by default